In [1]:
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import mlflow
from datetime import datetime,timezone
import json
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import re
import yfinance as yf
import time
import os

In [2]:
mlflow.set_tracking_uri("http://20.75.92.162:5000")

In [3]:
mlflow.set_experiment("Finance_Sentiment_Anlysis_RohitZ")

<Experiment: artifact_location='mlflow-artifacts:/989261207141256503', creation_time=1758457167218, experiment_id='989261207141256503', last_update_time=1758457167218, lifecycle_stage='active', name='Finance_Sentiment_Anlysis_RohitZ', tags={}>

In [ ]:
company_name = (input(f"Enter company name :"))

In [14]:
with mlflow.start_run(run_name = f"sentiment_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
    mlflow.log_param("company_name", company_name)

    model_name='gpt4o'
    llm = AzureChatOpenAI(model=model_name)

    ticker_system_prompt = """
    You are a precise financial symbol resolver. Given a company name,"
    "return its most common pubic stock ticker and exchange." 
    "If ambiguous or private, return your best guess with lower confidence.Respond as strict JSON with keys:"
    '{{\"company\",\"ticker\",\"exchange\",\"confidence\"}}. No markdown
    """

    ticker_user_prompt = "Company: {company}"

    prompt_version_ticker = "V1"

    mlflow.log_dict(
        {"prompt_id" : "ticker_resolver", "version" : prompt_version_ticker, "system_template" : ticker_system_prompt,
         "user_template": ticker_user_prompt},
         "prompts/ticker_prompt_template.json"
    )

    ticker_prompt = ChatPromptTemplate.from_messages([("system",ticker_system_prompt),("user",ticker_user_prompt)])

    ticker_llm = ticker_prompt | llm #| RunnableLambda(lambda ai: json.loads(ai.content))

    response_schemas = [
        ResponseSchema(name = "company_name", description = "Company name (string)"),
        ResponseSchema(name = "stock_code", description = "Stock ticker/symbol"),
        ResponseSchema(name = "newsdesc", description = "Short cohesive summary of recent news"),
        ResponseSchema(name = "sentiment", description = "One of: Positive, Negative, Neutral"),
        ResponseSchema(name = "people_names", description = "List of people names"),
        ResponseSchema(name = "places_names", description = "List of places"),
        ResponseSchema(name = "other_companies_referred", description = "List of other companies mentioned"),
        ResponseSchema(name = "related_industries", description = "List of related industries"),
        ResponseSchema(name = "market_implications", description = "Likely market reaction"),
        ResponseSchema(name = "confidence_score", description = "Confidence in [0,1] (float)")
    ]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()


    prompt_version_sentiment = "V1"

    sentiment_system_prompt = """
            "You are an financial analyst. You will receive latest news about a company, produce a concise, balanced sentiment report. "
            "Classify the overall sentiment, extract named entities and produce STRICT JSON exactly with the keys below."
            "No markdown or extra text-return ONLY the JSON object.\n{format_instructions}"
            """
    
    sentimet_user_prompt = """
            Company: {company}\nStock code: {ticker}\nAs-of (UTC): {as_of}\n\n"
            "Concise headlines/summary:\n{concise}\n\n"
            "Instructions:\n"
            "-'newsdesc' must be short, cohesive narrative summary of the recent news.\n"
            "-'sentiment' must be exactly one of: Positive, Negative, Neutral.\n"
            "-Extract 'people_names', 'places_names', 'other_companies_referred', 'related_indutries'.\n"
            "-'market_implications' should breifly state how markets/investors may react.\n"
            "-'confidence_score' in [0,1]."
        """

    sentiment_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",sentiment_system_prompt),
        (
            "user",sentimet_user_prompt)
    ]
    )

    mlflow.log_dict(
        {"prompt_id" : "sentiment_analysis", "version" : prompt_version_sentiment, "system_template" : sentiment_system_prompt,
         "user_template": sentimet_user_prompt},
         "prompts/sentiment_prompt_template.json"
    )

    chain = (
        RunnableLambda(lambda _:
                           {'company': company_name}
                           )
        | RunnablePassthrough.assign(t0_ticker=RunnableLambda(lambda _:time.perf_counter()))

        | RunnablePassthrough.assign(ticker_format_formatted=RunnableLambda(lambda x: [{"type": getattr(m, "type", ""), "content": m.content}
                                                                                       for m in ticker_prompt.format_messages(company = x["company"])]))
        
        | RunnablePassthrough.assign(ticker_ai = ticker_llm)

        | RunnablePassthrough.assign(
            ticker_info_llm = RunnableLambda(
                lambda x: (lambda txt: (json.loads(re.search(r"\{.*\}", txt, flags = re.DOTALL).group(0)) if re.search(r"\{.*\}", txt, flags = re.DOTALL) else json.loads(txt)))(
                    getattr(x["ticker_ai"], "content", str(x['ticker_ai']))
                )
            )
        )

        | RunnablePassthrough.assign(
            ticker_info=RunnableLambda(lambda x: x["ticker_info_llm"])
        )
        | RunnablePassthrough.assign(
            ticker=RunnableLambda(lambda x: (x['ticker_info'].get('ticker',"").strip() or "UNKNOWN")),
            company_canonical = RunnableLambda(lambda x: (x['ticker_info'].get("company") or x['company']))
        )
        
        | RunnablePassthrough.assign(
            news_items = RunnableLambda(
                lambda x: (
                    [
                        {
                        "headline": n['content'].get("title"),
                        "summary" : n['content'].get("summary"),
                        "url": n['content']['thumbnail']['originalUrl'],
                        "source": n['content'].get("provider")['displayName'],
                        "published_at" : (
                            datetime.fromtimestamp(n['content']["providerPublishTime"], tz=timezone.utc).isoformat() if n['content'].get("providerPublishTime") else None
                ),
                        } for n in (yf.Ticker(x['ticker']).news or [])
                        
                    ] or [{
                        "headline" : "No recent articles found.", "summary" : None, "url" : None, "source" : None, "published_at" : None
                    }]
                )
            )
        )

        | RunnablePassthrough.assign(
            concise=RunnableLambda(lambda x: ("\n".join(
                "-" + (it.get("headline") or "").strip() + (": " + it["summary"].strip() if it.get("summary") else "")
                for it in x['news_items']
            ))[:2000]
            ),
            as_of = RunnableLambda(lambda _: datetime.now(tz=timezone.utc).isoformat()),
            news_json=RunnableLambda(lambda x: json.dumps(x['news_items'], ensure_ascii=False, indent=2)),
            format_instructions=RunnableLambda(lambda _: format_instructions)
        )

        | RunnablePassthrough.assign(
            sentiment_prompt_formatted = RunnableLambda(
                lambda x: [{"type" : getattr(m, "type", ""), "content" : m.content}
                           for m in sentiment_prompt.format_messages(
                               format_instructions=x['format_instructions'],
                               as_of=x['as_of'], company = x['company_canonical'], ticker = x['ticker'],
                               concise = x['concise'][:2500]
                           )
                           ]
            )
        )

        | RunnablePassthrough.assign(
            sentiment_ai = (sentiment_prompt | llm)
        )

        | RunnablePassthrough.assign(
            report=RunnableLambda(lambda x:output_parser.invoke(getattr(x["sentiment_ai"], "content", str(x["sentiment_ai"]))))
        )
        | RunnableLambda(lambda x: {
            "company": x["company_canonical"],
            "stock_code": x['ticker'],
            "ticker_info" : x["ticker_info"],
            # "news": x["news_items"],
            "analysis" :{
                "company_name": x["report"].get("company_name", x["company_canonical"]),
                "stock_code": x["report"].get("stock_code", x["ticker"]),
                "newsdesc": x["report"].get("newsdesc", ""),
                "sentiment": x["report"].get("sentiment", ""),
                "people_names": x["report"].get("people_names", []) or [],
                "places_names": x["report"].get("places_names", []) or [],
                "other_companies_referred": x["report"].get("other_companies_referred", []) or [],
                "related_industries": x["report"].get("related_industries", []) or [],
                "market_implications": x["report"].get("market_implications", ""),
                "confidence_score": x["report"].get("confidence_score", 0.0),
            }


        })
        )
    
    result = chain.invoke({})
    mlflow.log_dict(result, "artifacts/final_output.json")


🏃 View run sentiment_20250921_201817 at: http://20.75.92.162:5000/#/experiments/989261207141256503/runs/2469d68b5ea84af296eeab1b8d788f08
🧪 View experiment at: http://20.75.92.162:5000/#/experiments/989261207141256503


In [15]:
result

{'company': 'Microsoft',
 'stock_code': 'MSFT',
 'ticker_info': {'company': 'Microsoft',
  'ticker': 'MSFT',
  'exchange': 'NASDAQ',
  'confidence': 'high'},
 'analysis': {'company_name': 'Microsoft',
  'stock_code': 'MSFT',
  'newsdesc': 'Microsoft, along with Amazon and Google, has advised its H-1B visa employees to remain in the U.S. amidst significant changes to visa applications proposed by the Trump administration.',
  'sentiment': 'Neutral',
  'people_names': 'Donald Trump',
  'places_names': 'United States',
  'other_companies_referred': 'Amazon, Google, AT&T, Seagate Technology, Asana',
  'related_industries': 'Technology, Immigration, Finance',
  'market_implications': 'Investors may remain cautious in the tech sector due to potential impacts of immigration policy changes.',
  'confidence_score': '0.7'}}